
# Introduction #

Welcome to the [**Petals to the Metal**](https://www.kaggle.com/c/tpu-getting-started) competition! In this competition, you’re challenged to build a machine learning model to classify 104 types of flowers based on their images.

In this tutorial notebook, you'll learn how to build an image classifier in Keras and train it on a [Tensor Processing Unit (TPU)](https://www.kaggle.com/docs/tpu). At the end, you'll have a complete project you can build off of with ideas of your own.

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:24px;">
    <strong>Fork This Notebook!</strong><br>
Create your own editable copy of this notebook by clicking on the <strong>Copy and Edit</strong> button in the top right corner.
</blockquote>


# Introduction #

[**Petals to the Metal**](https://www.kaggle.com/c/tpu-getting-started) yarışmasına hoş geldiniz! Bu yarışmada, 104 çiçek türünü görüntülerine göre sınıflandırmak için bir makine öğrenimi modeli oluşturmanız isteniyor.
Bu eğitici not defterinde, Keras'ta bir görüntü sınıflandırıcıyı nasıl oluşturacağınızı ve bunu bir [Tensor İşleme Birimi (TPU)](https://www.kaggle.com/docs/tpu) üzerinde nasıl eğiteceğinizi öğreneceksiniz. Sonunda, kendi fikirlerinizle inşa edebileceğiniz eksiksiz bir projeniz olacak.

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:24px;">
    <strong>Fork This Notebook!</strong><br>
Create your own editable copy of this notebook by clicking on the <strong>Copy and Edit</strong> button in the top right corner.
</blockquote>

# Step 1: Imports #

İlk önce python pakettlerini import ediyoruz.

In [ ]:
import math, re, os
import numpy as np
import tensorflow as tf

print("Tensorflow version " + tf.__version__)

# Step 2: Distribution Strategy #

A TPU has eight different *cores* and each of these cores acts as its own accelerator. (A TPU is sort of like having eight GPUs in one machine.) We tell TensorFlow how to make use of all these cores at once through a **distribution strategy**. Run the following cell to create the distribution strategy that we'll later apply to our model.

# Step 2: TPU Tanımlaması ve Kullanımı ile Dağıtım Stratejisi (Distribution Strategy) #

Bir TPU'nun *sekiz farklı çekirdeği* vardır ve bu çekirdeklerin her biri kendi hızlandırıcısı olarak işlev görür. (Bir TPU, bir makinede sekiz GPU'ya sahip olmak gibidir.) TensorFlow'a bir **Dağıtım Stratejisi (Distribution Strategy)** aracılığıyla tüm bu çekirdeklerin bir kerede nasıl kullanılacağını anlatıyoruz. Daha sonra modelimize uygulayacağımız dağıtım stratejisini oluşturmak için aşağıdaki hücreyi çalıştırın.


In [ ]:
#TPU'yu algılıyoruz, uygun olan dağıtım stratejisini (Distribution Strategy) döndürüyoruz
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

Sinir ağı modelimizi oluştururken dağıtım stratejisini kullanacağız. Ardından TensorFlow, her çekirdek için bir model olmak üzere sekiz farklı model *replika* oluşturarak eğitimi sekiz TPU çekirdeği arasında dağıtır.
# Step 3: Loading the Competition Data -  Yarışma Verilerinin Yüklenmesi #

## Get GCS Path - GCS Yolunu Alın ##

TPU'larla kullanıldığında veri kümelerinin bir [Google Cloud Storage paketinde](https://cloud.google.com/storage/) depolanması gerekir. Herhangi bir genel GCS paketindeki verileri, tıpkı `'/kaggle/input'`taki veriler gibi yolunu vererek kullanabilirsiniz. Aşağıdakiler, bu yarışmanın veri kümesi için GCS yolunu alacaktır.

In [ ]:
# Veri kümesi için GCS yolunu alacak
from kaggle_datasets import KaggleDatasets

GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
print(GCS_DS_PATH) # what do gcs paths look like?

You can use data from any public dataset here on Kaggle in just the same way. If you'd like to use data from one of your private datasets, see [here](https://www.kaggle.com/docs/tpu#tpu3pt5).

## Load Data - Veri yükle ##

TPU'larla kullanıldığında, veri kümeleri genellikle [TFRecords](https://www.kaggle.com/ryanholbrook/tfrecords-basics) olarak serileştirilir. Bu, her bir TPU çekirdeğine veri dağıtmak için uygun bir biçimdir. İşlem biraz uzun olduğu için veri setimiz için TFRecords'u okuyan hücreyi gizledik. TPU'larla kendi veri kümelerinizi kullanma konusunda bazı rehberlik için daha sonra geri dönebilirsiniz.

In [ ]:

IMAGE_SIZE = [512, 512]
GCS_PATH = GCS_DS_PATH + '/tfrecords-jpeg-512x512'
AUTO = tf.data.experimental.AUTOTUNE

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec') 

CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102


def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

## Create Data Pipelines - Veri İşlem Hatları Oluşturun ##

Bu son adımda, eğitim, doğrulama ve test bölümlerinin her biri için verimli bir veri hattı tanımlamak üzere `tf.data` API'sini kullanacağız.

In [ ]:

def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO)
    # statement in the next function (below), this happens essentially
    # for free on TPU. Data pipeline code is executed on the "CPU"
    # part of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_saturation(image, 0, 2)
    return image, label   

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec
    # files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))


Sonraki hücre, eğitim ve çıkarım sırasında Keras ile kullanacağımız veri kümelerini oluşturacaktır. Partilerin boyutunu TPU çekirdeği sayısına nasıl ölçeklendirdiğimize dikkat edin.

In [ ]:
# Define the batch size. This will be 16 with TPU off and 128 (=16*8) with TPU on
# Parti boyutunu tanımlayın. Bu, TPU kapalıyken 16 ve TPU açıkken 128 (=16*8) olacaktır.
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

ds_train = get_training_dataset()
ds_valid = get_validation_dataset()
ds_test = get_test_dataset()

print("Training:", ds_train)
print ("Validation:", ds_valid)
print("Test:", ds_test)

Bu veri kümeleri `tf.data.Dataset` nesneleridir. TensorFlow'daki bir veri kümesini bir *veri kaydı akışı (stream)* olarak düşünebilirsiniz. Eğitim ve doğrulama kümeleri, `(image, label)` çiftlerinin akışlarıdır.

In [ ]:
#Eğitim verilerinin boyutları

np.set_printoptions(threshold=15, linewidth=80)

print("Training data shapes:")
for image, label in ds_train.take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy())

Test seti, `(image, idnum)` çiftlerinin bir akışıdır; Buradaki `idnum`, daha sonra `csv` dosyası olarak gönderimimizi yaptığımızda kullanacağımız resme verilen benzersiz tanımlayıcıdır.

In [ ]:
# Test verilerinin Boyutları.
print("Test data shapes:")
for image, idnum in ds_test.take(3):
    print(image.numpy().shape, idnum.numpy().shape)
print("Test data IDs:", idnum.numpy().astype('U')) # U=unicode string

# Step 4: Explore Data - Verileri Keşfedin #

Veri setindeki bazı resimlere bir göz atalım.

In [ ]:

from matplotlib import pyplot as plt

def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case,
                                     # these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is
    # the case for test data)
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)
    
def display_batch_of_images(databatch, predictions=None):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
        
    # auto-squaring: this will drop data that does not fit into square
    # or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_flower(image, title, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()


def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

Başka bir yardımcı işlevimizle bir veri kümesinden tek bir toplu resim görüntüleyebilirsiniz. Sonraki hücre, veri kümesini 20 görüntüden oluşan toplu bir yineleyiciye dönüştürecektir.

In [ ]:
ds_iter = iter(ds_train.unbatch().batch(20))

Akıştaki bir sonraki grubu çıkarmak ve yardımcı işlevle görüntülemek için Python `next` işlevini kullanın.

In [ ]:
one_batch = next(ds_iter)
display_batch_of_images(one_batch)

`ds_iter` ve `one_batch`'i ayrı hücrelerde tanımlayarak, yeni bir toplu resim görmek için yalnızca yukarıdaki hücreyi yeniden çalıştırmanız gerekir.

# Step 5: Define Model - Modeli Tanımlayın #

Artık görüntüleri sınıflandırmak için bir sinir ağı oluşturmaya hazırız! **Transfer öğrenimi (transfer learning)** olarak bilinen yöntemi kullanacağız. Aktarım öğrenimi ile, yeni bir veri kümesine önde başlamak için önceden eğitilmiş bir modelin bir bölümünü yeniden kullanırsınız.


Bu eğitim için, [ImageNet](http://image-net.org/) üzerinde önceden eğitilmiş **VGG16** adlı bir model kullanacağız. Daha sonra, Keras'ta bulunan [diğer modelleri](https://www.tensorflow.org/api_docs/python/tf/keras/applications) denemek isteyebilirsiniz. ([Xception](https://www.tensorflow.org/api_docs/python/tf/keras/applications/Xception) kötü bir seçim olmaz.)

Daha önce oluşturduğumuz dağıtım stratejisi bir [bağlam yöneticisi](https://docs.python.org/3/reference/compound_stmts.html#with), `strategy.scope` içerir. Bu bağlam yöneticisi, TensorFlow'a eğitim çalışmasının sekiz TPU çekirdeği arasında nasıl bölüneceğini söyler. TensorFlow'u bir TPU ile kullanırken, modelinizi bir `strategy.scope()` bağlamında tanımlamanız önemlidir.


In [ ]:
EPOCHS = 12

with strategy.scope():
    pretrained_model = tf.keras.applications.VGG16(
        weights='imagenet',
        include_top=False ,
        input_shape=[*IMAGE_SIZE, 3]
    )
    pretrained_model.trainable = False
    
    model = tf.keras.Sequential([
        # To a base pretrained on ImageNet to extract features from images...
        pretrained_model,
        # ... attach a new head to act as a classifier.
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])

Kayıp ve metriklerin `'sparse_categorical'`sürümleri, bunun gibi ikiden fazla etiket içeren bir sınıflandırma görevi için uygundur.

In [ ]:
model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

model.summary()

# Step 6: Training - Eğitim #

## Learning Rate Schedule - Öğrenme Hızı Çizelgesi ##

Bu ağı özel bir öğrenme oranı programıyla eğiteceğiz.

In [ ]:

# Learning Rate Schedule for Fine Tuning #
def exponential_lr(epoch,
                   start_lr = 0.00001, min_lr = 0.00001, max_lr = 0.00005,
                   rampup_epochs = 5, sustain_epochs = 0,
                   exp_decay = 0.8):

    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        # linear increase from start to rampup_epochs
        if epoch < rampup_epochs:
            lr = ((max_lr - start_lr) /
                  rampup_epochs * epoch + start_lr)
        # constant max_lr during sustain_epochs
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        # exponential decay towards min_lr
        else:
            lr = ((max_lr - min_lr) *
                  exp_decay**(epoch - rampup_epochs - sustain_epochs) +
                  min_lr)
        return lr
    return lr(epoch,
              start_lr,
              min_lr,
              max_lr,
              rampup_epochs,
              sustain_epochs,
              exp_decay)

lr_callback = tf.keras.callbacks.LearningRateScheduler(exponential_lr, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [exponential_lr(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

## Fit Model  - Modeli Eğiteceğiz ##

Ve şimdi modeli eğitmeye hazırız. Birkaç parametre tanımladıktan sonra, gitmeye hazırız!

In [ ]:
# Define training epochs
EPOCHS = 12
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[lr_callback],
)

Bu sonraki hücre, eğitim sırasında kaybın ve ölçümlerin nasıl ilerlediğini gösterir. Neyse ki, birleşiyor!

In [ ]:
display_training_curves(
    history.history['loss'],
    history.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    history.history['sparse_categorical_accuracy'],
    history.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)

# Step 7: Evaluate Predictions - Tahminleri Değerlendirin #

Test setinde son tahminlerinizi yapmadan önce, modelinizin tahminlerini doğrulama setinde değerlendirmek iyi bir fikirdir. Bu, eğitimdeki sorunları teşhis etmenize veya modelinizin iyileştirilebileceği yollar önermenize yardımcı olabilir. İki yaygın doğrulama yöntemine bakacağız: **karışıklık matrisinin (confusion matrix)** çizilmesi ve **görsel doğrulama (visual validation)**.

In [ ]:

import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(CLASSES)))
    ax.set_xticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(CLASSES)))
    ax.set_yticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()
    
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

##  Confusion Matrix - Karışıklık Matrisi ##

Bir [karışıklık matrisi (confusion matrix)](https://en.wikipedia.org/wiki/Confusion_matrix), tahmini sınıfına göre tablo haline getirilmiş bir görüntünün gerçek sınıfını gösterir. Bir sınıflandırıcının performansını değerlendirmek için sahip olduğunuz en iyi araçlardan biridir.

Aşağıdaki hücre, doğrulama verileri üzerinde bazı işlemler yapar ve ardından [`scikit-learn`](https://scikit-learn.org/stable/index.html) içinde bulunan `confusion_matrix` işleviyle matrisi oluşturur.

In [ ]:
cmdataset = get_validation_dataset(ordered=True)
images_ds = cmdataset.map(lambda image, label: image)
labels_ds = cmdataset.map(lambda image, label: label).unbatch()

cm_correct_labels = next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy()
cm_probabilities = model.predict(images_ds)
cm_predictions = np.argmax(cm_probabilities, axis=-1)

labels = range(len(CLASSES))
cmat = confusion_matrix(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
)
cmat = (cmat.T / cmat.sum(axis=1)).T # normalize

[F1 puanı(score)](https://en.wikipedia.org/wiki/F1_score) veya [kesinlik ve hatırlama (precision and recall)](https://en.wikipedia.org/wiki/Precision_and_recall) gibi metriklere aşina olabilirsiniz. Bu hücre, bu metrikleri hesaplayacak ve bunları karışıklık matrisinin bir grafiğiyle görüntüleyecektir. (Bu metrikler, Scikit-learn modülü `sklearn.metrics` içinde tanımlanmıştır; bunları sizin için yardımcı komut dosyasına aktardık.)

In [ ]:
score = f1_score(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
    average='macro',
)
precision = precision_score(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
    average='macro',
)
recall = recall_score(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
    average='macro',
)
display_confusion_matrix(cmat, score, precision, recall)

## Visual Validation - Görsel Doğrulama ##

Doğrulama setinden bazı örneklere bakmak ve modelinizin hangi sınıfı tahmin ettiğini görmek de yardımcı olabilir. Bu, modelinizin sorun yaşadığı görüntü türlerindeki kalıpları ortaya çıkarmaya yardımcı olabilir.

Bu hücre, doğrulama setini bir seferde 20 görüntü gösterecek şekilde ayarlayacaktır - isterseniz bunu daha fazla veya daha az görüntülenecek şekilde değiştirebilirsiniz.

In [ ]:
dataset = get_validation_dataset()
dataset = dataset.unbatch().batch(20)
batch = iter(dataset)

Ve burada tahmin edilen türleriyle bir dizi çiçek var. Başka bir küme görmek için hücreyi tekrar çalıştırın.

In [ ]:
images, labels = next(batch)
probabilities = model.predict(images)
predictions = np.argmax(probabilities, axis=-1)
display_batch_of_images((images, labels), predictions)

# Step 8: Make Test Predictions - Test Tahminleri Yapın #

Her şeyden memnun kaldığınızda, test setinde tahminler yapmaya hazırsınız.

In [ ]:
test_ds = get_test_dataset(ordered=True)

print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

Bir `submission.csv` dosyası oluşturacağız. Bu dosya, skor tablosunda puanınızı almak için göndereceğiniz dosyadır.

In [ ]:
print('Generating submission.csv file...')

# Get image ids from test set and convert to unicode
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

# Write the submission file
np.savetxt(
    'submission.csv',
    np.rec.fromarrays([test_ids, predictions]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Look at the first few predictions
!head submission.csv

# Step 9: Make a submission #

If you haven't already, create your own editable copy of this notebook by clicking on the **Copy and Edit** button in the top right corner. Then, submit to the competition by following these steps:

1. Begin by clicking on the blue **Save Version** button in the top right corner of the window.  This will generate a pop-up window.  
2. Ensure that the **Save and Run All** option is selected, and then click on the blue **Save** button.
3. This generates a window in the bottom left corner of the notebook.  After it has finished running, click on the number to the right of the **Save Version** button.  This pulls up a list of versions on the right of the screen.  Click on the ellipsis **(...)** to the right of the most recent version, and select **Open in Viewer**.  This brings you into view mode of the same page. You will need to scroll down to get back to these instructions.
4. Click on the **Output** tab on the right of the screen.  Then, click on the file you would like to submit, and click on the blue **Submit** button to submit your results to the leaderboard.

You have now successfully submitted to the competition!

If you want to keep working to improve your performance, select the blue **Edit** button in the top right of the screen. Then you can change your code and repeat the process. There's a lot of room to improve, and you will climb up the leaderboard as you work.


---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161321) to chat with other Learners.*